# **Import Library**

In [0]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# **Import File**

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
df_one = pd.read_csv('/content/drive/My Drive/work/nectec/dataset7.csv',parse_dates=['Date'])
with pd.ExcelFile('/content/drive/My Drive/work/nectec/dataSet4 (2)-newFromITaaS.xlsx') as reader:
  df_two = pd.read_excel(reader, sheet_name=reader.sheet_names[1])

# **Clean Data**

In [44]:
def sum_data(df_one,df_two):
  df = pd.merge(df_one, df_two, on='Item No', how='inner')
  df['sum_price'] = df['Price'] * df['Quantity']
  df = df[['Receipt No','Item No','Price','Quantity','Date','Description','sum_price']]
  df = df.groupby(['Item No','Date']).sum().reset_index()
  df = df[['Receipt No','Item No','Date','sum_price','Quantity']]
  return df
df_test_two = sum_data(df_one,df_two)['Item No'].unique().tolist()
len(df_test_two)
sum_data(df_one,df_two)

,Receipt No,Item No,Date,sum_price,Quantity
0,94097602587,1111001A,2018-01-01,110320.0,3152.0
1,124086775957,1111001A,2018-01-02,126140.0,3604.0
2,37836447721,1111001A,2018-01-03,36715.0,1049.0
3,22617610866,1111001A,2018-01-04,23800.0,680.0
4,17664752529,1111001A,2018-01-05,22225.0,635.0
...,...,...,...,...,...
127927,80134040,3999004A,2018-09-07,516.0,4.0
127928,130044853,3999004A,2018-09-22,129.0,1.0
127929,80100758,3999005A,2018-03-23,358.0,2.0
127930,140073841,3999005A,2018-09-21,358.0,2.0


# **Function**

In [45]:
def choose_year(df,year):
  df['year'] = df['Date'].dt.year
  df = df[df['year'] == year]
  # df = df.groupby(['Receipt No','Item No']).sum().reset_index()[['Receipt No','Item No','sum_price','Quantity']]
  df = df.groupby(['Item No']).sum().reset_index()[['Item No','sum_price','Quantity']]
  return df
# choose_year(sum_data(df_one,df_two),2018)

def choose_income_quality(df,choose_income_quality,percent_data):

  df = df.sort_values(f"{choose_income_quality}",ascending=False)
  df[f'total_{choose_income_quality}'] = df[f'{choose_income_quality}'].sum()
  df[f'percent_{choose_income_quality}'] = df[f'{choose_income_quality}']/df[f'total_{choose_income_quality}'] * 100
  df[f'cumulative_{choose_income_quality}'] = df[f'percent_{choose_income_quality}'].cumsum()
  df
  df = df.reset_index()
  df = df.reset_index()
  df = df.rename(columns={"level_0": "percent"})
  df['percent'] = df['percent'] + 1
  df['percent'] = df['percent']/len(df) * 100
  df = df[df['percent'] < percent_data]

  return df

df = choose_income_quality(choose_year(sum_data(df_one,df_two),2018),'sum_price',20)
df
# df[df['Item No'] == '1122201A']
# df_test = choose_income_quality(choose_year(sum_data(df_one,df_two),2018),'Quantity',80)


,percent,index,Item No,sum_price,Quantity,total_sum_price,percent_sum_price,cumulative_sum_price
0,0.057703,0,1111001A,4447905.0,127083.0,75222440.84,5.913003,5.913003
1,0.115407,326,1199008A,3322890.0,21438.0,75222440.84,4.417418,10.330421
2,0.173110,125,1122201A,3317195.0,94777.0,75222440.84,4.409848,14.740269
3,0.230814,2,1111002A,2987705.0,85363.0,75222440.84,3.971827,18.712096
4,0.288517,147,1124060A,2522600.0,12613.0,75222440.84,3.353521,22.065616
...,...,...,...,...,...,...,...,...
341,19.734564,992,3259011A,35520.0,888.0,75222440.84,0.047220,85.653095
342,19.792268,918,3251094A,35415.0,787.0,75222440.84,0.047080,85.700176
343,19.849971,365,1612033A,35280.0,588.0,75222440.84,0.046901,85.747077
344,19.907675,1517,3541014A,35100.0,117.0,75222440.84,0.046662,85.793738


# **table**

In [10]:
import plotly.graph_objects as go

headerColor = 'mediumblue'
rowEvenColor = 'white'
rowOddColor = 'lightsteelblue'

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Item No</b>','<b>price</b>','<b>quantity</b>','<b>cumulative</b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      df['Item No'].tolist(),
      df['sum_price'].tolist(),
      df['Quantity'].tolist(),
      df['cumulative_sum_price'].tolist()],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.show()

# **Masket Basket**

In [12]:
import datetime
time_start = datetime.datetime.now()
df_basket = (choose_income_quality(choose_year(sum_data(df_one,df_two),2018),'sum_price',80)
          .groupby(['Receipt No', 'Item No'])['sum_price']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Receipt No'))

basket_sets = df_basket.applymap(lambda x: min(max(0,x), 1))
frequent_itemsets = apriori(basket_sets, min_support=0.0000001, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules
rules["antecedents"] = rules["antecedents"].apply(lambda x: list(x)[0]).astype("unicode")
rules["consequents"] = rules["consequents"].apply(lambda x: list(x)[0]).astype("unicode")
df_market_basket = rules[['antecedents','consequents','antecedent support','consequent support','support','confidence','lift','leverage','conviction']]








time_end = datetime.datetime.now()
print(time_end - time_start)

KeyboardInterrupt: ignored

In [0]:
df_market_basket

In [0]:
import itertools
import networkx as nx
df_market_basket['node'] = list(zip(df_market_basket['antecedents'],  df_market_basket['consequents']))

G = nx.Graph()
edges_fig_4 = df_market_basket['node'].tolist()
edges_fig_4
G.add_edges_from(edges_fig_4)
cliques = nx.find_cliques(G)
length_node = 2
cliques3 = set(sum([list(itertools.combinations(set(clq), length_node)) for clq in cliques if len(clq)>length_node],[]))
# cliques3
print(list(cliques3))

In [0]:
df_market_basket

# **Export Csv**

In [0]:
df_market_basket
df_market_basket.to_csv('market_basket.csv')